In [2]:
!pip install -q kaggle

In [3]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kavyajshah24","key":"14ce48092b35803ec19be9a9b8ffc9d7"}'}

In [4]:
mkdir ~/.kaggle

In [5]:
cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets list

ref                                                           title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ruchikakumbhar/obesity-prediction                             Obesity Prediction Dataset                         58KB  2025-01-14 05:31:18           1244         27  1.0              
anandshaw2001/netflix-movies-and-tv-shows                     Netflix Movies and TV Shows                         1MB  2025-01-03 10:33:01           4280        118  1.0              
raedaddala/top-500-600-movies-of-each-year-from-1960-to-2024  30,000+ Movies, 60+ Years of Data, Rich Metadata    4MB  2025-01-05 17:17:05            901         28  1.0              
preetigupta004/cancer-issue                                   Cancer issue      

In [8]:
!kaggle competitions download -c induction-task


 97% 195M/201M [00:01<00:00, 169MB/s]
100% 201M/201M [00:01<00:00, 124MB/s]


In [9]:
!unzip induction-task.zip

Archive:  induction-task.zip
  inflating: Data/Test/image_1.jpg   
  inflating: Data/Test/image_10.jpg  
  inflating: Data/Test/image_100.jpg  
  inflating: Data/Test/image_101.jpg  
  inflating: Data/Test/image_102.jpg  
  inflating: Data/Test/image_103.jpg  
  inflating: Data/Test/image_104.jpg  
  inflating: Data/Test/image_105.jpg  
  inflating: Data/Test/image_106.jpg  
  inflating: Data/Test/image_107.jpg  
  inflating: Data/Test/image_108.jpg  
  inflating: Data/Test/image_109.jpg  
  inflating: Data/Test/image_11.jpg  
  inflating: Data/Test/image_110.jpg  
  inflating: Data/Test/image_111.jpg  
  inflating: Data/Test/image_112.jpg  
  inflating: Data/Test/image_113.jpg  
  inflating: Data/Test/image_114.jpg  
  inflating: Data/Test/image_115.jpg  
  inflating: Data/Test/image_116.jpg  
  inflating: Data/Test/image_117.jpg  
  inflating: Data/Test/image_118.jpg  
  inflating: Data/Test/image_119.jpg  
  inflating: Data/Test/image_12.jpg  
  inflating: Data/Test/image_120.jpg  


In [10]:
!ls /content/Data


Test  Train


In [11]:
TRAIN_DIR = "/content/Data/Train"

In [12]:
import os

# Check dataset path
TRAIN_DIR = "/content/Data/Train"  # Update based on your structure
print(os.listdir(TRAIN_DIR))  # Should list the class folders

['AI', 'Real']


In [13]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [14]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

dataset_directory = '/content/Data/Train'

image_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

image_dataset = datasets.ImageFolder(root=dataset_directory, transform=image_transforms)

train_split = int(0.8 * len(image_dataset))
val_split = len(image_dataset) - train_split

train_data, val_data = random_split(image_dataset, [train_split, val_split])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

print(f"Training dataset size: {len(train_data)}")
print(f"Validation dataset size: {len(val_data)}")

In [15]:
class BasicCNN(nn.Module):
    def __init__(self, num_classes):
        super(BasicCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(32 * 16 * 16, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.model(x)

In [26]:
num_classes = 2
model = BasicCNN(num_classes=num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

In [22]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1/10, Loss: 0.2927
Epoch 2/10, Loss: 0.0994
Epoch 3/10, Loss: 0.0483
Epoch 4/10, Loss: 0.0342
Epoch 5/10, Loss: 0.0149
Epoch 6/10, Loss: 0.0049
Epoch 7/10, Loss: 0.0026
Epoch 8/10, Loss: 0.0015
Epoch 9/10, Loss: 0.0011
Epoch 10/10, Loss: 0.0008


In [23]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


Validation Accuracy: 97.52%


In [25]:
import os
import torch
from torchvision import transforms
from PIL import Image
import pandas as pd

test_dataset_path = '/content/Data/Test'
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
def process_test_data(test_dataset_path, transform):
    image_paths = []
    images = []

    for filename in os.listdir(test_dataset_path):
        filepath = os.path.join(test_dataset_path, filename)
        if os.path.isfile(filepath):
            image_paths.append(filepath)
            img = Image.open(filepath).convert('RGB')
            img = transform(img)
            images.append(img)

    return torch.stack(images), image_paths

test_images, image_paths = process_test_data(test_dataset_path, transform)


In [ ]:
model.eval()

device = torch.device('cpu')
model.to(device)

def predict(model, test_images, image_paths):
    test_images = test_images.to(device)
    results = []

    with torch.no_grad():
        outputs = model(test_images)
        _, predicted = torch.max(outputs, 1)
    class_mapping = {0: 'AI', 1: 'Real'}
    for img_path, pred in zip(image_paths, predicted):
        label = class_mapping[pred.item()]
        results.append({'Image': os.path.basename(img_path), 'Prediction': label})
    return results

predictions = predict(model, test_images, image_paths)

In [ ]:
df = pd.DataFrame(predictions)
output_csv_path = '/content/Task1.csv'
df.to_csv(output_csv_path, index=False)
print(f"Predictions saved to {output_csv_path}")

Predictions saved to /content/Task1.csv


In [ ]:
import pandas as pd

file_path = 'Task1.csv'
df = pd.read_csv(file_path)
df['Image'] = df['Image'].str.replace('.jpg', '', regex=False)
output_file_path = 'Task1_modified.csv'
df.to_csv(output_file_path, index=False)
print(f"Modified file saved to {output_file_path}")

Modified file saved to Task1_modified.csv
